<a href="https://colab.research.google.com/github/arnaldojr/previsao-covid-19/blob/master/EP4_IME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#nao esquece das classicas
import numpy as np

In [ ]:
df = pd.read_csv('/content/Covid_filtrada.csv', encoding='windows-1250', sep=';')
df.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (127) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID_Paciente,25OH-Vitamina D Total,Ácido fólico,Ácido Úrico,AgHBs,Albumina,ALT (TGP),Amilase,Anti-HBs,Anti-HCV,"Aspecto, urina",AST (TGO),Basófilos,Bastonetes,BE arterial,BE venoso,Bilirrubina Direta,Bilirrubina Indireta,Bilirrubina Total,"Bilirrubina, urina",BNP,"C. Cetônicos, urina",Cálcio,Cálcio Ionizável,Cálculo p/afrodescendente CKD-EPI,Cálculo p/afrodescendente MDRD,Cálculo p/năo afrodescendente CKD-EPI,Cálculo p/năo afrodescendente MDRD,Células Epiteliais,CHCM,"Cilindros, urina",CK,CK-MB,Cloro,Clostridium difficile,Colesterol total,"Cor, urina",Creatinina,"Cultura Automatizada, 3Ş Amostra",Cultura de Urina,...,Plaquetas,pO2 arterial,pO2 venoso,Potássio,"Potássio, sangue","Potássio, sangue total",Promielócitos,Proteína C-Reativa,Proteínas,"Proteínas, urina",RDW,Sat O2 arterial,Sat O2 venoso,Segmentados,Siderofilina,Sódio,"Sódio, sangue",Tempo de Protrombina,TP/INR,Triglicérides,Troponina,TSH,TTPA,TTPA - Paciente/Normal,Uréia,Urina Tipo I,"Urobilinogęnio,urina",Vancomicina,VCM,VHS,Vitamina B-12,VLDL-Colesterol,Volume plaquetário médio,X-Coronavírus (2019-nCoV),"X-Covid 19, Anticorpos IgA","X-Covid 19, Anticorpos IgA, índice","X-Covid 19,Antic IgM/IgG,teste rápido-Interpret","X-Covid 19, por PCR","X-Covid 19, Anticorpos IgG","X-Covid 19, Anticorpos IgM"
0,00177290279939FB33386B29198C450E11/jun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
1,003051C9B19101D1C10C5DC65438401701/jun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
2,003051C9B19101D1C10C5DC65438401704/jun,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1,NaN,NaN
3,003051C9B19101D1C10C5DC65438401712/jun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1,1.0,1.0
4,003F1F4C194763E4A00FD809AF5FA6AA01/abr,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.replace({np.nan: -1})
df.head()

,ID_Paciente,25OH-Vitamina D Total,Ácido fólico,Ácido Úrico,AgHBs,Albumina,ALT (TGP),Amilase,Anti-HBs,Anti-HCV,"Aspecto, urina",AST (TGO),Basófilos,Bastonetes,BE arterial,BE venoso,Bilirrubina Direta,Bilirrubina Indireta,Bilirrubina Total,"Bilirrubina, urina",BNP,"C. Cetônicos, urina",Cálcio,Cálcio Ionizável,Cálculo p/afrodescendente CKD-EPI,Cálculo p/afrodescendente MDRD,Cálculo p/năo afrodescendente CKD-EPI,Cálculo p/năo afrodescendente MDRD,Células Epiteliais,CHCM,"Cilindros, urina",CK,CK-MB,Cloro,Clostridium difficile,Colesterol total,"Cor, urina",Creatinina,"Cultura Automatizada, 3Ş Amostra",Cultura de Urina,...,Plaquetas,pO2 arterial,pO2 venoso,Potássio,"Potássio, sangue","Potássio, sangue total",Promielócitos,Proteína C-Reativa,Proteínas,"Proteínas, urina",RDW,Sat O2 arterial,Sat O2 venoso,Segmentados,Siderofilina,Sódio,"Sódio, sangue",Tempo de Protrombina,TP/INR,Triglicérides,Troponina,TSH,TTPA,TTPA - Paciente/Normal,Uréia,Urina Tipo I,"Urobilinogęnio,urina",Vancomicina,VCM,VHS,Vitamina B-12,VLDL-Colesterol,Volume plaquetário médio,X-Coronavírus (2019-nCoV),"X-Covid 19, Anticorpos IgA","X-Covid 19, Anticorpos IgA, índice","X-Covid 19,Antic IgM/IgG,teste rápido-Interpret","X-Covid 19, por PCR","X-Covid 19, Anticorpos IgG","X-Covid 19, Anticorpos IgM"
0,00177290279939FB33386B29198C450E11/jun,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
1,003051C9B19101D1C10C5DC65438401701/jun,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
2,003051C9B19101D1C10C5DC65438401704/jun,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
3,003051C9B19101D1C10C5DC65438401712/jun,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0
4,003F1F4C194763E4A00FD809AF5FA6AA01/abr,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [ ]:
df.shape

(10142, 130)

# fazendo o tratamento do dataset customizado

In [ ]:
import torch 
from torch import nn, optim

from torch.utils.data import DataLoader
from torch.utils.data import Dataset


#nao esquece das classicas
import numpy as np


# Dicionario de hiperparametros
args = {
    'batch_size': 20,
    'num_workers': 4,
    'hidden_size': 128,
    'lr': 1e-4,
    'weight_decay': 5e-4,
    'num_epochs': 30
}

# testa GPU
if torch.cuda.is_available():
  args['device'] = torch.device('cuda')
else:
  args['device'] = torch.device('cpu')
print(args['device'])


cuda


In [ ]:
#separação de treino e teste

torch.manual_seed(1)
indices = torch.randperm(len(df)).tolist()

train_size = int(0.8*len(df))
df_train = df.iloc[indices[:train_size]]
df_test = df.iloc[indices[train_size:]]

print(len(df_train),len(df_test))
display(df_test.head())

df_train.to_csv('covid_train.csv', index=False)
df_test.to_csv('covid_test.csv', index=False)

8113 2029


,ID_Paciente,25OH-Vitamina D Total,Ácido fólico,Ácido Úrico,AgHBs,Albumina,ALT (TGP),Amilase,Anti-HBs,Anti-HCV,"Aspecto, urina",AST (TGO),Basófilos,Bastonetes,BE arterial,BE venoso,Bilirrubina Direta,Bilirrubina Indireta,Bilirrubina Total,"Bilirrubina, urina",BNP,"C. Cetônicos, urina",Cálcio,Cálcio Ionizável,Cálculo p/afrodescendente CKD-EPI,Cálculo p/afrodescendente MDRD,Cálculo p/năo afrodescendente CKD-EPI,Cálculo p/năo afrodescendente MDRD,Células Epiteliais,CHCM,"Cilindros, urina",CK,CK-MB,Cloro,Clostridium difficile,Colesterol total,"Cor, urina",Creatinina,"Cultura Automatizada, 3Ş Amostra",Cultura de Urina,...,Plaquetas,pO2 arterial,pO2 venoso,Potássio,"Potássio, sangue","Potássio, sangue total",Promielócitos,Proteína C-Reativa,Proteínas,"Proteínas, urina",RDW,Sat O2 arterial,Sat O2 venoso,Segmentados,Siderofilina,Sódio,"Sódio, sangue",Tempo de Protrombina,TP/INR,Triglicérides,Troponina,TSH,TTPA,TTPA - Paciente/Normal,Uréia,Urina Tipo I,"Urobilinogęnio,urina",Vancomicina,VCM,VHS,Vitamina B-12,VLDL-Colesterol,Volume plaquetário médio,X-Coronavírus (2019-nCoV),"X-Covid 19, Anticorpos IgA","X-Covid 19, Anticorpos IgA, índice","X-Covid 19,Antic IgM/IgG,teste rápido-Interpret","X-Covid 19, por PCR","X-Covid 19, Anticorpos IgG","X-Covid 19, Anticorpos IgM"
5888,99FD4B0A36D9C7138ABD7295F8F6B54810/mai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1.0,1.0
3852,64F21C80F5E771AD68EAC44B55B0877108/mai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
2908,494B98F9CF96777FB53FFBD43BCC28D516/jun,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4972,7DBD70BBD04BCF581CB56516E15DBA7101/mai,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
1210,23801D369E037F1809EE82954B7D6AC716/abr,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0


In [ ]:
!ls
print(df_train.shape)
print(df_test.shape)


Covid_filtrada.csv  covid_test.csv  covid_train.csv  sample_data
(8113, 130)
(2029, 130)


In [ ]:
# Criar a classe Dataset

class CovidSet(Dataset):
  def __init__(self, csv_path):
    self.dados = pd.read_csv(csv_path).to_numpy()

  def __getitem__(self, idx):

    sample = self.dados[idx][2:126]
    label = self.dados[idx][130:]

    #Converte para tensor
    sample = torch.from_numpy(sample.astype(np.float32))
    np.nan_to_num(sample, copy=False)
    label = torch.from_numpy(label.astype(np.float32))
    np.nan_to_num(label, copy=False)
    return sample, label

  
  def __len__(self):
    
    return len(self.dados)

In [ ]:
train_set = CovidSet('covid_train.csv')
test_set = CovidSet('covid_test.csv')


dado, rotulo = train_set[0]
print(rotulo, dado)

tensor([]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (127) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
train_loader = DataLoader(train_set,
                          batch_size=args['batch_size'],
                          shuffle=True,
                          num_workers=args['num_workers'])

test_loader = DataLoader(test_set,
                          batch_size=args['batch_size'],
                          shuffle=True,
                          num_workers=args['num_workers'])

In [ ]:
for batch in train_loader:
  dado, rotulo = batch
  print(dado.size(), rotulo.size())

  break

torch.Size([20, 124]) torch.Size([20, 0])


In [ ]:
#Implementando ao MLP
#pega o ultima feito e altera

# Implementar o MLP

class MLP(nn.Module):

  def __init__(self, input_size, hidden_size, output_size): 
    super(MLP, self).__init__()
    
    self.features = nn.Sequential(
                    nn.Linear(input_size,hidden_size),
                    nn.ReLU(),
                    nn.Linear(hidden_size, hidden_size),
                    nn.ReLU()
                    )
    self.out = nn.Linear(hidden_size, output_size)
    #self.softmax = nn.Softmax() nao faz sentido usar softmax

  def forward(self, X):
    
    feature = self.features(X)
    output = self.out(feature)

    return output

input_size = len(train_set[0][0]) #42
hidden_size = args['hidden_size']
output_size = 1 #quantas bikes foram alugadas, é o que será predito.

net = MLP(input_size, hidden_size, output_size).to(args['device'])

In [ ]:
#criterio de perda
criterion = nn.L1Loss().to(args['device'])
optimizer = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])

In [ ]:
#fluxo de treinamento
def train(train_loader,net, epoch):
  net.train() # garante modo de treinamento 
  epoch_loss = []
  for batch in train_loader:
    
    dado, rotulo = batch

    # Cast na GPU
    dado   = dado.to(args['device'])
    rotulo = rotulo.to(args['device'])

    # Forward 
    pred = net(dado)
    loss = criterion(pred, rotulo)
    epoch_loss.append(loss.cpu().data)

    # Backward
    loss.backward()
    optimizer.step()

  epoch_loss = np.asarray(epoch_loss)
  print("--------treinamento-----")
  print("Epoca %d, Loss: %.4f +\- %.4f" % (epoch, epoch_loss.mean(), epoch_loss.std()))
  return epoch_loss.mean()

In [ ]:
#fluxo de validação / teste
def test(test_loader,net, epoch):
  #tem que fazer isso para rodar os testes
  net.eval()
  with torch.no_grad(): 
    #o resto é igual
    epoch_loss = []
    for batch in test_loader:
      
      dado, rotulo = batch

      # Cast na GPU
      dado   = dado.to(args['device'])
      rotulo = rotulo.to(args['device'])

      # Forward 
      pred = net(dado)
      loss = criterion(pred, rotulo)
      epoch_loss.append(loss.cpu().data)

    epoch_loss = np.asarray(epoch_loss)
    print("--------validação-----")
    print("Epoca %d, Loss: %.4f +\- %.4f" % (epoch, epoch_loss.mean(), epoch_loss.std()))
    return epoch_loss.mean()

In [ ]:
train_losses, test_losses = [], []
for epoch in range(args['num_epochs']):
  train_losses.append(train(train_loader,net,epoch))
  test_losses.append(test(test_loader,net,epoch))
  print("--------------------")


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([20, 0])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


--------treinamento-----
Epoca 0, Loss: nan +\- nan


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([13, 0])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


--------validação-----
Epoca 0, Loss: nan +\- nan
--------------------


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:88: UserWarning: Using a target size (torch.Size([9, 0])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


--------treinamento-----
Epoca 1, Loss: nan +\- nan
--------validação-----
Epoca 1, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 2, Loss: nan +\- nan
--------validação-----
Epoca 2, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 3, Loss: nan +\- nan
--------validação-----
Epoca 3, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 4, Loss: nan +\- nan
--------validação-----
Epoca 4, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 5, Loss: nan +\- nan
--------validação-----
Epoca 5, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 6, Loss: nan +\- nan
--------validação-----
Epoca 6, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 7, Loss: nan +\- nan
--------validação-----
Epoca 7, Loss: nan +\- nan
--------------------
--------treinamento-----
Epoca 8, Loss: nan +\- nan
--------validação-----
Epoca 8, Loss: nan +\- nan
--------------------
--------treiname

In [ ]:
Xtest = torch.stack([tup[0] for tup in test_set])
Xtest = Xtest.to(args['device'])

ytest = torch.stack([tup[1] for tup in test_set])
ypred = net(Xtest).cpu().data

data = torch.cat((ytest, ypred), axis=1)

df_results = pd.DataFrame(data, columns=['ypred', 'ytest'])
df_results.head(20)

ValueError: ignored

In [ ]:
plt.figure(figsize=(20, 9))
plt.plot(train_losses, label='Train')
plt.plot(test_losses, label='Test', linewidth=3, alpha=0.5)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Convergence', fontsize=16)
plt.legend()
plt.show()

In [ ]:
print(train_losses)